In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skimage.measure as measure
import skimage.morphology as morphology
import skimage.segmentation as segmentation

In [ ]:
# перевод изображения из формата bgr в rgb и отображение на экране
def show_plt(img):
    b,g,r = cv2.split(img)  
    img2 = cv2.merge([r,g,b])
    plt.imshow(img2)

In [ ]:
# перевод изображения из формата bgr в rgb и сохранение в файл 
def save_plt(filename, img):
    b,g,r = cv2.split(img)  
    img2 = cv2.merge([r,g,b])
    plt.imsave(filename, img2)

In [ ]:
# вычисление евклидова расстояния между векторами
def eucl_dist(vec_1, vec_2):  
    return np.sqrt(np.sum((vec_1 - vec_2) ** 2))

In [ ]:
# образцы цветов, используемых в фишках; взяты с одного из изображений
col_sample = np.array([  
    [18, 30, 162], # red
    [60, 49, 57],  # blue
    [ 12, 134, 194], # yellow
    [32, 45, 61] # black
])

In [ ]:
# функция возвращает номер в массиве col_sample цвета, наиболее близкого к входным данным 
# по евклидовой метрике
def get_color(curr_col):  
    return np.argmin(np.array([
            eucl_dist(curr_col, col_sample[0]),
            eucl_dist(curr_col, col_sample[1]), 
            eucl_dist(curr_col, col_sample[2]),
            eucl_dist(curr_col, col_sample[3])]))

In [ ]:
# элемент массива описывает форму сегментов каждого из 3-х цветов на фишках указанных справа номеров 
# (есть несколько пар фишек, типы линий на которых совпадают)
# порядок цветов сегментов: [красный, синий, желтый]
# номером 1 обозначается короткая дуга, 2 - длинная дуга, 3 - прямая линия
shapes_sample = np.array([[2, 2, 1], # 1, 10
                          [1, 3, 1], # 2
                          [1, 1, 1], # 3
                          [2, 3, 2], # 4
                          [3, 1, 1], # 5
                          [2, 2, 3], # 6
                          [2, 1, 2], # 7, 8
                          [3, 2, 2], # 9
                         ])

In [ ]:
path = # insert path to data 
img_name = # insert image name
part_of_task = # insert number of task: 1 or 2

# для удобства дальнейшей обработки считываем изображение в цветном и черно-белом вариантах
img_gray = cv2.imread(path + img_name, cv2.IMREAD_GRAYSCALE)
img_color = cv2.imread(path + img_name, cv2.IMREAD_COLOR)

In [ ]:
# бинаризация изображения с использованием порогового значения
_, threshold = cv2.threshold(img_gray, 100, 1, type=cv2.THRESH_BINARY)
threshold = np.logical_not(threshold)
plt.imshow(threshold, cmap='gray')

Построим маску: пикселям каждой компоненты связности присвоим свое уникальное значение, одинаковое внутри одной компоненты связности и различное для разных компонент. В аргумент num записывается количество компонент связности в изображении, значения элеметов матрицы лежат в диапазоне от 1 до num, элементам фона присваивается значение 0.

In [ ]:
labels, num = measure.label(threshold, return_num=True)
plt.imshow(labels, cmap='gray')

Для каждой компоненты связности построим выпуклую оболочку

In [ ]:
convex_img = np.zeros(threshold.shape)
for i in range(1, num + 1):
    convex_img += (morphology.convex_hull_image(labels == i)).astype(np.int64)
plt.imshow(convex_img, cmap='gray')

Чтобы сгладить неровности по краям, применим операцию эрозии. На форму шестиугольников это не повлияет.

In [ ]:
convex_img = morphology.erosion(convex_img, morphology.star(5))
convex_img = (convex_img * 255).astype(np.uint8)
plt.imshow(convex_img, cmap='gray')

Теперь, когда границы фигур сглажены, можно применить операцию распознавания контуров.

In [ ]:
cnts = cv2.findContours(convex_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]

Начнем обрабатывать полученные контуры:

In [ ]:
obj_list = []  # в список будем вносить массивы вершин шестиугольников
img_color_copy = img_color.copy()
for c in cnts:  # цикл по всем найденным контурам
    peri = cv2.arcLength(c, True)  # вычислим периметр фигуры
    if(peri > 100):  # если фигура не слишком маленькая, чтобы быть фишкой
        approx = cv2.approxPolyDP(c, 0.025 * peri, True)  # аппроксимируем границы
        # возвращаемое значение функции - массив угловых точек фигуры
        flag = True 
        if(len(approx) == 6):  # если углов у фигуры 6, то перейдем к дальнейшему рассмотрению
            edges_list = []  # приведем координаты вершин к более удобному формату
            for i in range(6):
                edges_list.append([approx[i][0, 0], approx[i][0, 1]])
            edges_list = np.array(edges_list)
            for i in range(1, 7):
                dot_1 = edges_list[i % 6]
                dot_0 = edges_list[(i - 1) % 6]
                if np.abs(eucl_dist(dot_1, dot_0) - peri / 6) > 0.1 * peri:
                # объект может являться фишкой, если длина каждой из шести его сторон 
                # примерно равна 1/6 периметра
                    flag = False
            if flag:  # если все шесть сторон удовлетворяют условию, то считаем объект фишкой
                # нанесем контуры и углы найденной фишки на изображение и инкрементируем значение счетчика
                cv2.drawContours(img_color_copy, [approx], -1, (255, 0, 0), 4)
                for n, i in enumerate(approx):
                    cv2.circle(img_color_copy, (i[0, 0], i[0, 1]), 3, (0,0,255), -1)
                obj_list.append(np.array(edges_list).copy())
show_plt(img_color_copy)        

In [ ]:
# массив коэффициентов, применяющийся при поиске точки, лежащей внутри цветного сегмента на фишке
coeff_arr = np.array([[0.5, 0.5, 1.0, 0.0], 
                       [0.5, 0.5, 0.9, 0.1],
                       [0.4, 0.6, 1.0, 0.0],
                       [0.6, 0.4, 1.0, 0.0],
                      [0.4, 0.6, 0.9, 0.1],
                      [0.6, 0.4, 0.9, 0.1]
                     ])

In [ ]:
img_color_copy = img_color.copy()
for object in obj_list:
    col_list = [[] for i in range(3)]
    dot_list = []
    center = np.mean(object, axis=0)
    for i in range(1, 7):  # цикл по 6-ти ребрам фигуры
        num_try = 0
        # проверяем, пока не попадем в точку, цвет в которой не будет похож на черный
        while((num_try == 0 or get_color(curr_col) == 3) and num_try < coeff_arr.shape[0]):
            # ищем точку примерно в центре ребра
            edge_center = (coeff_arr[num_try, 0] * object[i % 6] + 
                           coeff_arr[num_try, 1] * object[i - 1]
                          ).astype(np.uint64)
            # немного сдвигаемся к центру фишки
            curr_dot = (coeff_arr[num_try, 2] * edge_center + 
                        coeff_arr[num_try, 3] * center).astype(np.uint64)
            # фиксируем цвет найденной точки
            curr_col = img_color[curr_dot[1]][curr_dot[0]]
            num_try += 1
        if(num_try == coeff_arr.shape[0]):
            # если перебрали все точки из указанного в массиве диапазона, но цвет в каждой из них близок
            # к черному, то предположим, что через данное ребро проходит синяя линия
            # это кажется логичным, тк синий цвет наиболее близок к черному, а в случае ошибки в списках 
            # распределения точек по цветам окажется отличное от 2 число элементов, и это будет исправлено 
            # на следующем шаге алгоритма
            col_ind = 1
        else:
            col_ind = get_color(curr_col)
        dot_list.append(curr_dot)
        col_list[col_ind].append(i - 1)
    dot_list = np.array(dot_list)
    # если в списке точек, отнесенных к какому-то цвету, оказалось меньше 2-x, то нужно 
    # найти наиболее похожего кандидата в списке, в котором больше 2-х элементов,
    # и перераспределить точки в списках
    # будем повторять процедуру, пока в каждом списке не окажется по 2 точки
    loop_flag = True
    while(loop_flag):
        for i in range(3):
            if(len(col_list[i]) < 2):
                not_enough = i
                loop_flag = False
            elif(len(col_list[i]) > 2):
                too_much = i
        if(not loop_flag):
            much_dots = dot_list[col_list[too_much]]
            dist_list = []
            for dot in much_dots:
                dist_list.append(eucl_dist(img_color[dot[1]][dot[0]], col_sample[not_enough]))
            ind = np.argmin(np.array(dist_list))
            col_list[not_enough].append(col_list[too_much][ind])
            col_list[too_much].remove(col_list[too_much][ind])
            loop_flag = True
        else:
            loop_flag = False
    col_list = np.array(col_list)
    
    shape_list = []
    for i in range(3):
        # для каждого из 3-х цветов найдем число ребер шестиугольника, лежащих между двумя ребрами,
        # через которые проходит сегмент этого цвета; это однозначно  определит форму сегмента
        diff = min(col_list[i, 1] - col_list[i, 0], col_list[i, 0] + 6 - col_list[i, 1])
        shape_list.append(diff)
    # в соответствии с приведенным массивом по форме сегментов 3-х цветов определим номер фишки
    shape_num = np.argmax(np.sum(np.logical_not(shapes_sample - shape_list), axis=1)) + 1
    if(shape_num == 8):
        shape_num = 9
    if(shape_num == 1):
        col_list = (col_list + (6 - col_list[2, 0])) % 6
        if(np.min(col_list[0, :]) > np.min(col_list[1, :])): 
            # если начало красного сегмента имеет больший номер, чем начало синего
            shape_num = 10
    if(shape_num == 7):
        col_list = (col_list + (6 - col_list[1, 0])) % 6
        if(np.min(col_list[0, :]) > np.min(col_list[2, :])): 
            # если начало красного сегмента имеет больший номер, чем начало желтого
            shape_num = 8
            
    # отобразим квадрат с номером фишки на копии изображения, если выполняется вторая часть задания
    if(part_of_task == 2):
        cv2.rectangle(img_color_copy, (object[0, 0], object[0, 1]), (object[0, 0] + 45, object[0, 1] + 45), 
                      (255,255,255), -1)
        cv2.putText(img_color_copy, str(shape_num) ,(object[0, 0] + 5, object[0, 1] + 35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, -1, 2)
    else:
    # напечатаем найденный номер, если выполняется первая часть задания
        print("фишка №", end='')
        print(shape_num)
if(part_of_task == 2):
    show_plt(img_color_copy)